In [ ]:
### Uses 20 day and 50 day moving average as a momentum strategy. If the 50 day is larger than the 20 day M.A. then we buy (the 20 day MA will increase), and vice versa. 

import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.factors import SimpleMovingAverage
import quantopian.optimize as opt
from quantopian.pipeline.experimental import risk_loading_pipeline

#initializer that gets the data about SPY
def initialize(context):
    context.spy = symbol('SPY')
    #makes it so only long buys and sell are possible
    set_long_only()
    
#handles the data, this is what makes the buys and the sells
def handle_data(context, data):
   #history of the SPY, price from 50 days every minute
    hist = data.history(context.spy, fields="price", bar_count=50, frequency="1m")
    sma_50 = hist.mean()              #fifty day moving average
    sma_20 = hist[-20:].mean()     #20 day moving average

    #sees what orders we already have
    open_orders = get_open_orders()
    
   # if 50 MA is greater than 20 MA, and we do not have those orders, buy all
    if sma_50 > sma_20:
        if context.spy not in open_orders:
            order_target_percent(context.spy, 1.0)
    
  #else, get rid of it 
    elif sma_20 > sma_50:
        if context.spy in open_orders:
            order_target_percent(context.spy, 0.0)   

##overall this code wasn’t great, it basically followed the benchmark. 
##additionally, couldn’t figure out how to use history correctly to get the moving average to start #right away and not after 50 days.